#  Import Body Map from Infopad csv file

#### Last updated on : 3rd Oct 2020  -  Amar Jaiswal

In [24]:
es_base_url = 'http://localhost:9200'
headers = {'Content-Type': 'application/json'}

In [25]:
# auto sugestion with tab
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline 

In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as datetime

import requests
import json

import os
os.getcwd()

'/lhome/amardj/ISM/python_code/es_scripts/case_base_patient_scripts'

In [27]:
sys_date = datetime.datetime.today().strftime( '%Y%m%d')
sys_date = f'{sys_date}' 
sys_date

'20201021'

In [28]:
data_dir = f'./data/{sys_date}/'

In [29]:
body_main_file = f'{data_dir}body_map.csv'

In [30]:
raw_csv = pd.read_csv( body_main_file, delimiter=';')

In [31]:
raw_csv.head()

,Brukernavn,Journal ID,Fødselsår,Forside 0 0,Forside 0 1,Forside 0 2,Forside 0 3,Forside 0 4,Forside 0 5,Forside 0 6,...,Bakside 7 5,Bakside 7 6,Bakside 7 7,Bakside 7 8,Bakside 7 9,Bakside 7 10,Bakside 7 11,Bakside 7 12,Bakside 7 13,Bakside 7 14
0,support10,1,1963,,,,,,,,...,,,,,,,,,,
1,support10,2,1965,,,,,,,,...,,,,,,,,,,
2,support23,3,1950,,,,,,,,...,,,,,,,,,,
3,support10,5,1983,,,,,,,,...,,,,,,,,,,
4,support10,6,1958,,,,,,,,...,,,,,,,,,,


In [32]:
raw = raw_csv.rename(columns={'Fødselsår': 'birthYear', 'Brukernavn':'clinicId', 'Journal ID':'journalId'})

In [33]:
exclude_patients_from_clinic = raw[
    (raw.clinicId != 'Fredrik Granviken') 
    & (raw.clinicId != 'Anita Formo Bones')
    & (raw.clinicId != 'Ingebrigt Meisingset')
    & (raw.clinicId != 'Testinstitutt')
    & (raw.clinicId != 'support32')
    & (raw.clinicId != 'support3')
] #"Testinstitutt", "support32","support3"

In [34]:
exclude_patients_from_clinic.shape #(*, 243)

(63, 243)

In [35]:
exclude_patients_from_clinic.head()

,clinicId,journalId,birthYear,Forside 0 0,Forside 0 1,Forside 0 2,Forside 0 3,Forside 0 4,Forside 0 5,Forside 0 6,...,Bakside 7 5,Bakside 7 6,Bakside 7 7,Bakside 7 8,Bakside 7 9,Bakside 7 10,Bakside 7 11,Bakside 7 12,Bakside 7 13,Bakside 7 14
0,support10,1,1963,,,,,,,,...,,,,,,,,,,
1,support10,2,1965,,,,,,,,...,,,,,,,,,,
2,support23,3,1950,,,,,,,,...,,,,,,,,,,
3,support10,5,1983,,,,,,,,...,,,,,,,,,,
4,support10,6,1958,,,,,,,,...,,,,,,,,,,


In [36]:
es = []

for index, df in exclude_patients_from_clinic.iterrows():
    #print(index)

    
    df_1_row = df.replace(' ',np.nan).dropna(axis=0,how="all")
    
    patient_id = df_1_row[0].replace(' ','_').lower() + '_'+str(df_1_row[1]) +'_'+str(df_1_row[2])
            

    print(patient_id)

    patient = {}
    patient_details = {}
    body_map = {}

    patient.update({'patientId': patient_id}) 

    front = []
    back = []

    for val in df_1_row.index[3:].tolist():
        tokens = str(val).split(' ')

        if tokens[0] == 'Forside':     
            front.append({ 'x':int(tokens[1]), 'y':int(tokens[2])})

        if tokens[0] == 'Bakside':
            back.append({ 'x':int(tokens[1]), 'y':int(tokens[2])})

    body_map.update({'front':front})
    body_map.update({'back':back})

    patient_details.update({'body_map':body_map})
    patient.update({'patientDetails':patient_details})

    es.append(patient)
    
es;

support10_1_1963
support10_2_1965
support23_3_1950
support10_5_1983
support10_6_1958
support10_7_1956
support10_8_1967
support10_9_1966
support10_10_1964
support10_11_1966
support10_12_1950
support10_13_1977
support24_101_1984
support24_102_1978
support26_1001_1950
support27_1002_1990
support11_1808_1969
support11_1810_1989
support28_2000_1973
support28_2002_1981
support28_2005_1956
support28_2006_1957
support9_2748_1996
support7_3816_1984
support19_5191_1968
support7_5655_1992
support15_6245_1969
support9_8017_1973
support9_8092_1982
support6_8853_1968
support9_9145_1975
support9_9327_1955
support5_11785_1956
support5_17504_1973
support6_24778_1984
support5_26296_1962
support13_29310_2002
support5_30160_1992
support13_31894_1956
support12_31903_1972
support13_31909_1995
support12_31910_1968
support5_31911_1995
support5_31912_1971
support5_31959_1969
support12_31992_1961
support5_32015_1965
support12_32035_1969
support5_32110_1959
support5_32111_1998
support13_32177_1961
support31_3235

In [37]:
json.dumps(es)

'[{"patientId": "support10_1_1963", "patientDetails": {"body_map": {"front": [{"x": 1, "y": 8}, {"x": 6, "y": 8}], "back": [{"x": 1, "y": 8}, {"x": 3, "y": 2}, {"x": 4, "y": 2}, {"x": 6, "y": 8}]}}}, {"patientId": "support10_2_1965", "patientDetails": {"body_map": {"front": [{"x": 2, "y": 2}, {"x": 5, "y": 2}, {"x": 5, "y": 3}], "back": [{"x": 2, "y": 3}, {"x": 3, "y": 2}, {"x": 3, "y": 3}, {"x": 4, "y": 2}, {"x": 4, "y": 3}, {"x": 4, "y": 8}, {"x": 4, "y": 10}, {"x": 4, "y": 11}, {"x": 4, "y": 12}, {"x": 4, "y": 13}, {"x": 5, "y": 3}, {"x": 5, "y": 8}]}}}, {"patientId": "support23_3_1950", "patientDetails": {"body_map": {"front": [{"x": 2, "y": 3}, {"x": 3, "y": 10}, {"x": 3, "y": 14}, {"x": 4, "y": 10}, {"x": 4, "y": 14}], "back": [{"x": 3, "y": 6}, {"x": 3, "y": 10}, {"x": 3, "y": 14}, {"x": 4, "y": 6}, {"x": 4, "y": 10}, {"x": 4, "y": 14}, {"x": 5, "y": 3}]}}}, {"patientId": "support10_5_1983", "patientDetails": {"body_map": {"front": [{"x": 2, "y": 2}, {"x": 2, "y": 3}, {"x": 3, "

In [38]:
with open(f'{data_dir}body_map.json', 'w', encoding='utf-8') as f:
    json.dump(es, f, ensure_ascii=False, indent=0)

# 
# 
# 

# Elasticsearch Operations

## Code for getting a patient by patientId from the ES

# 
## Set the elasticsearch (es) index name
## 

In [39]:
es_index = 'manual_patients'

# 

In [40]:
def get_all_docs(es_index=es_index):
    response = requests.post(es_base_url+'/'+es_index+'/_search?pretty', headers=headers, data='{}')
    return response.json()

In [41]:
def get_doc_by_pid(pid, es_index=es_index,):
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.post(es_base_url+'/'+es_index+'/_search?pretty', headers=headers, data=data)
    return response.json()

In [42]:
def get_id_for_pid( pid, es_index=es_index):
    params = (('filter_path', 'hits.hits._source'))
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.get(f'{es_base_url}/{es_index}/_search?filter_path=hits.hits._id', headers=headers, data=data)
    response_json = response.json()
    return response_json.get('hits', {}).get('hits', [{}])[0].get('_id')

In [43]:
get_id_for_pid('vidar_guttormsen_30160_1992')

# 

# 

## Code for getting only the list of patient ids from the ES

In [44]:
def update_es_doc_for_id( _id, json_doc, es_index=es_index):
    upsert_doc = json.dumps({'doc': json_doc, 'doc_as_upsert': True})
    response = requests.post(f'{es_base_url}/{es_index}/_update/{_id}?pretty', headers=headers, data=upsert_doc)
    return response.json()

In [45]:
def upadate_es_index(body_maps_list=es, es_index=es_index):
    for body_map in body_maps_list:
        pid = body_map.get('patientId')
        print(pid)
        _id = get_id_for_pid( pid, es_index)
        print(_id)
        
        if _id is None:
            print('_id was None')
        else :
            print('Upserting the pid:',pid, ' and _id: ', _id)
            update_es_doc_for_id(_id, body_map, es_index)

In [46]:
upadate_es_index( es, 'manual_patients')

support10_1_1963
ZRdLTHUBPCTLibsE6V94
Upserting the pid: support10_1_1963  and _id:  ZRdLTHUBPCTLibsE6V94
support10_2_1965
ZhdLTHUBPCTLibsE6V-y
Upserting the pid: support10_2_1965  and _id:  ZhdLTHUBPCTLibsE6V-y
support23_3_1950
fBdLTHUBPCTLibsE6l9f
Upserting the pid: support23_3_1950  and _id:  fBdLTHUBPCTLibsE6l9f
support10_5_1983
ZxdLTHUBPCTLibsE6V-7
Upserting the pid: support10_5_1983  and _id:  ZxdLTHUBPCTLibsE6V-7
support10_6_1958
aBdLTHUBPCTLibsE6V_C
Upserting the pid: support10_6_1958  and _id:  aBdLTHUBPCTLibsE6V_C
support10_7_1956
aRdLTHUBPCTLibsE6V_N
Upserting the pid: support10_7_1956  and _id:  aRdLTHUBPCTLibsE6V_N
support10_8_1967
ahdLTHUBPCTLibsE6V_W
Upserting the pid: support10_8_1967  and _id:  ahdLTHUBPCTLibsE6V_W
support10_9_1966
axdLTHUBPCTLibsE6V_f
Upserting the pid: support10_9_1966  and _id:  axdLTHUBPCTLibsE6V_f
support10_10_1964
bBdLTHUBPCTLibsE6V_o
Upserting the pid: support10_10_1964  and _id:  bBdLTHUBPCTLibsE6V_o
support10_11_1966
bRdLTHUBPCTLibsE6V_w
Upser